In [1]:
from langchain.retrievers import EnsembleRetriever # 여러 retriever를 입력으로 받아 처리
from langchain_community.retrievers import BM25Retriever  #TF-IDF 계열의 검색 알고리즘
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

In [2]:
doc_list_1 = [
    "프렌치 불독: 사교적이고 친근한 성격을 가지고 있으며, 조용하고 집에서 지내기에 적합 합니다",
    "비글: 호기심이 많고, 에너지가 넘치며, 사냥 본능이 강합니다. ",
    "독일 셰퍼드: 용감하고 지능적이며, 충성심이 강합니다",
    "포메라니안: 활발하고 호기심이 많으며, 주인에게 매우 애정적입니다",
    "치와와: 작지만 용감하고, 주인에게 깊은 애정을 보입니다",
    "보더 콜리:	매우 지능적이고 학습 능력이 뛰어나며, 에너지가 많아 많은 운동이 필요합니다 "
]

# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)
bm25_retriever.k = 2

doc_list_2 = [
    "프렌치 불독: 열에 약하므로 주의가 필요합니다",
    "비글: 가족과 잘 지내며, 아이들과 노는 것을 좋아합니다.",
    "독일 셰퍼드: 경찰견이나 구조견으로 많이 활용되며, 적절한 훈련과 운동이 필요합니다.",
    "포메라니안: 털이 풍성하므로 정기적인 그루밍이 필요합니다.",
    "치와와: 다른 동물이나 낯선 사람에게는 조심스러울 수 있습니다.",
    "보더 콜리: 목축견으로서의 본능이 강하며, 다양한 트릭과 명령을 쉽게 배울 수 있습니다."
]

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
faiss_vectorstore = FAISS.from_texts(
    doc_list_2, embedding, metadatas=[{"source": 2}] * len(doc_list_2)
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5] #retriever 가중치 설정
)

C:\Users\user\AppData\Local\Temp\ipykernel_22536\3260847929.py:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\user\anaconda3\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
query = "충성심이 강한 강아지는?"
ensemble_result = ensemble_retriever.get_relevant_documents(query)
bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]\n", ensemble_result, end="\n\n")
print("[BM25 Retriever]\n", bm25_result, end="\n\n")
print("[FAISS Retriever]\n", faiss_result, end="\n\n")

[Ensemble Retriever]
 [Document(metadata={'source': 1}, page_content='독일 셰퍼드: 용감하고 지능적이며, 충성심이 강합니다'), Document(id='5ef20afe-712e-4874-9611-bc914f4598a3', metadata={'source': 2}, page_content='독일 셰퍼드: 경찰견이나 구조견으로 많이 활용되며, 적절한 훈련과 운동이 필요합니다.'), Document(metadata={'source': 1}, page_content='보더 콜리:\t매우 지능적이고 학습 능력이 뛰어나며, 에너지가 많아 많은 운동이 필요합니다 '), Document(id='9d77a88c-70b8-4850-9834-0ceff1874e1f', metadata={'source': 2}, page_content='프렌치 불독: 열에 약하므로 주의가 필요합니다')]

[BM25 Retriever]
 [Document(metadata={'source': 1}, page_content='독일 셰퍼드: 용감하고 지능적이며, 충성심이 강합니다'), Document(metadata={'source': 1}, page_content='보더 콜리:\t매우 지능적이고 학습 능력이 뛰어나며, 에너지가 많아 많은 운동이 필요합니다 ')]

[FAISS Retriever]
 [Document(id='5ef20afe-712e-4874-9611-bc914f4598a3', metadata={'source': 2}, page_content='독일 셰퍼드: 경찰견이나 구조견으로 많이 활용되며, 적절한 훈련과 운동이 필요합니다.'), Document(id='9d77a88c-70b8-4850-9834-0ceff1874e1f', metadata={'source': 2}, page_content='프렌치 불독: 열에 약하므로 주의가 필요합니다')]



C:\Users\user\AppData\Local\Temp\ipykernel_22536\1599854441.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ensemble_result = ensemble_retriever.get_relevant_documents(query)


In [4]:
query = "지능적인 강아지는?"
ensemble_result = ensemble_retriever.get_relevant_documents(query)
bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]\n", ensemble_result, end="\n\n")
print("[BM25 Retriever]\n", bm25_result, end="\n\n")
print("[FAISS Retriever]\n", faiss_result, end="\n\n")

[Ensemble Retriever]
 [Document(metadata={'source': 1}, page_content='보더 콜리:\t매우 지능적이고 학습 능력이 뛰어나며, 에너지가 많아 많은 운동이 필요합니다 '), Document(id='5ef20afe-712e-4874-9611-bc914f4598a3', metadata={'source': 2}, page_content='독일 셰퍼드: 경찰견이나 구조견으로 많이 활용되며, 적절한 훈련과 운동이 필요합니다.'), Document(metadata={'source': 1}, page_content='치와와: 작지만 용감하고, 주인에게 깊은 애정을 보입니다'), Document(id='592474f8-8e36-448f-a1dc-9fb613559f89', metadata={'source': 2}, page_content='비글: 가족과 잘 지내며, 아이들과 노는 것을 좋아합니다.')]

[BM25 Retriever]
 [Document(metadata={'source': 1}, page_content='보더 콜리:\t매우 지능적이고 학습 능력이 뛰어나며, 에너지가 많아 많은 운동이 필요합니다 '), Document(metadata={'source': 1}, page_content='치와와: 작지만 용감하고, 주인에게 깊은 애정을 보입니다')]

[FAISS Retriever]
 [Document(id='5ef20afe-712e-4874-9611-bc914f4598a3', metadata={'source': 2}, page_content='독일 셰퍼드: 경찰견이나 구조견으로 많이 활용되며, 적절한 훈련과 운동이 필요합니다.'), Document(id='592474f8-8e36-448f-a1dc-9fb613559f89', metadata={'source': 2}, page_content='비글: 가족과 잘 지내며, 아이들과 노는 것을 좋아합니다.')]

